In [ ]:
###########################
# Imports and Setup
###########################
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import base64

# Configure OS routines
import os

# Configure data manipulation
import numpy as np
import pandas as pd

# Import CRUD module
from CRUD_Python_Module import AnimalShelter

# Import helper modules
from data_helpers import normalize_dataframe, bucket_categories
from rescue_filters import apply_rescue_filter
from dashboard_auth import validate_credentials, get_auth_error_message, is_authenticated


###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Read all documents from database
df_raw = pd.DataFrame.from_records(db.read({}))

# Remove MongoDB _id column (ObjectID type causes issues with dash_table)
df_raw.drop(columns=['_id'], inplace=True)

# Normalize the dataframe (create age_weeks, sex, intact_status, valid_coords columns)
df = normalize_dataframe(df_raw)


###########################
# Dashboard Layout / View
###########################
app = JupyterDash(__name__, suppress_callback_exceptions=True)

# Add Leaflet CSS for map rendering
app.css.append_css({
    'external_url': 'https://unpkg.com/leaflet@1.9.4/dist/leaflet.css'
})

# Load and encode logo
image_filename = 'Grazioso-Salvare-Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Main Dashboard Layout
dashboard_layout = html.Div(id='dashboard-content', children=[
    # Header with logo and branding
    html.Div([
        html.A([
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                     alt='Grazioso Salvare Logo',
                     style={'height': '80px', 'display': 'block', 'margin': '0 auto'})
        ], href='https://www.snhu.edu', target='_blank'),
        html.H1('Grazioso Salvare Animal Rescue Dashboard',
                style={'textAlign': 'center', 'color': '#2c3e50', 'marginTop': '20px'}),
        html.P('Dashboard by Rick Goshen',
               style={'textAlign': 'center', 'fontStyle': 'italic', 'color': '#7f8c8d'}),
        html.P('CS 340 - Client/Server Development',
               style={'textAlign': 'center', 'color': '#95a5a6', 'fontSize': '14px'}),
    ], style={'padding': '20px', 'backgroundColor': '#ecf0f1', 'borderRadius': '10px', 'marginBottom': '20px'}),

    html.Hr(),

    # Filter Controls
    html.Div([
        html.H3('Select Rescue Type Filter:', style={'color': '#34495e'}),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': ' Water Rescue (Labrador, Chesapeake Bay Retriever, Newfoundland - Intact Female, 26-156 weeks)',
                 'value': 'water'},
                {'label': ' Mountain/Wilderness Rescue (German Shepherd, Alaskan Malamute, Old English Sheepdog, Siberian Husky, Rottweiler - Intact Male, 26-156 weeks)',
                 'value': 'mountain'},
                {'label': ' Disaster Rescue or Individual Tracking (Doberman Pinscher, German Shepherd, Golden Retriever, Bloodhound, Rottweiler - Intact Male, 20-300 weeks)',
                 'value': 'disaster'},
                {'label': ' Reset (Show All Animals)', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'block', 'marginBottom': '10px'},
            style={'padding': '15px'}
        )
    ], style={'padding': '20px', 'backgroundColor': '#f8f9fa', 'borderRadius': '10px', 'marginBottom': '20px'}),

    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False,
                  "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',
        selected_rows=[0],
        sort_action='native',
        page_action='native',
        page_current=0,
        page_size=10,
        style_cell={
            'textAlign': 'left',
            'minWidth': '100px',
            'width': '150px',
            'maxWidth': '200px',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
        },
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        tooltip_data=[
            {
                column: {'value': str(value), 'type': 'markdown'}
                for column, value in row.items()
            } for row in df.to_dict('records')
        ],
        tooltip_duration=None
    ),

    html.Br(),
    html.Hr(),

    # Charts side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Main App Layout with both login and dashboard always present
app.layout = html.Div([
    dcc.Store(id='auth-state', data={'authenticated': False}),
    dcc.Store(id='previous-page', data=0),  # Track previous page for detecting actual changes
    
    # Login Container (always in DOM, visibility toggled)
    html.Div(id='login-container', children=[
        html.Div([
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                     alt='Grazioso Salvare Logo',
                     style={'height': '100px', 'display': 'block', 'margin': '20px auto'}),
            html.H2('Grazioso Salvare Dashboard Login',
                    style={'textAlign': 'center', 'color': '#2c3e50'}),
            html.Hr(),
            html.Div([
                html.Label('Username:', style={
                           'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Input(id='username-input', type='text', placeholder='Enter username',
                          style={'width': '100%', 'padding': '10px', 'marginBottom': '15px'}),
                html.Label('Password:', style={
                           'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Input(id='password-input', type='password', placeholder='Enter password',
                          style={'width': '100%', 'padding': '10px', 'marginBottom': '15px'}),
                html.Button('Login', id='login-button', n_clicks=0,
                            style={'width': '100%', 'padding': '10px', 'backgroundColor': '#3498db',
                                   'color': 'white', 'border': 'none', 'borderRadius': '5px',
                                   'fontSize': '16px', 'cursor': 'pointer'}),
                html.Div(id='login-error',
                        style={'color': 'red', 'marginTop': '15px', 'textAlign': 'center'})
            ], style={'maxWidth': '400px', 'margin': '0 auto', 'padding': '30px',
                      'backgroundColor': '#ecf0f1', 'borderRadius': '10px'})
        ], style={'padding': '50px'})
    ], style={'display': 'block'}),
    
    # Dashboard Container (always in DOM, visibility toggled)
    html.Div(id='dashboard-container', children=dashboard_layout, style={'display': 'none'})
])


#############################################
# Interaction Between Components / Controller
#############################################

# Authentication callback - validates credentials and updates auth state + error message
@app.callback(
    [Output('auth-state', 'data'),
     Output('login-error', 'children')],
    [Input('login-button', 'n_clicks')],
    [State('username-input', 'value'),
     State('password-input', 'value')]
)
def authenticate_user(n_clicks, username, password):
    """Handle user authentication."""
    if n_clicks == 0:
        return {'authenticated': False}, ''

    # Validate credentials (dashboard_auth module handles None/empty checks)
    if validate_credentials(username, password):
        return {'authenticated': True}, ''
    else:
        error_msg = get_auth_error_message(username, password)
        return {'authenticated': False}, error_msg


# Toggle visibility between login and dashboard
@app.callback(
    [Output('login-container', 'style'),
     Output('dashboard-container', 'style')],
    [Input('auth-state', 'data')]
)
def toggle_screens(auth_state):
    """Show dashboard or login screen based on authentication state."""
    if is_authenticated(auth_state):
        # Hide login, show dashboard
        return {'display': 'none'}, {'display': 'block'}
    else:
        # Show login, hide dashboard
        return {'display': 'block'}, {'display': 'none'}


# Filter callback - updates data table, resets pagination, and resets row selection
@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'page_current'),
     Output('datatable-id', 'selected_rows')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    """Filter the data table and reset pagination/selection when filter changes."""
    try:
        # Apply the appropriate filter
        filtered_df = apply_rescue_filter(df, filter_type)
        # Return filtered data, reset to page 0, and select first row
        return filtered_df.to_dict('records'), 0, [0]
    except Exception as e:
        print(f"Error in filter callback: {e}")
        # Return full dataset on error, reset to page 0, select first row
        return df.to_dict('records'), 0, [0]


# Clear selection callback - resets row selection when page changes (not on initial load)
@app.callback(
    [Output('datatable-id', 'selected_rows', allow_duplicate=True),
     Output('previous-page', 'data')],
    [Input('datatable-id', 'page_current')],
    [State('previous-page', 'data'),
     State('datatable-id', 'selected_rows')],
    prevent_initial_call=True
)
def clear_selection_on_page_change(current_page, previous_page, current_selection):
    """Clear row selection when user navigates to a different page."""
    # Only clear if this is an actual page change (not initial load or filter change)
    if previous_page != current_page:
        # Clear selection and update previous page tracker
        return [], current_page
    else:
        # No change, keep current selection
        return current_selection, current_page


# Chart callback - displays outcome type distribution
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    """Update the outcome type distribution chart based on filtered data."""
    try:
        # Use datatable-id.data as fallback if derived_virtual_data is None
        if viewData is None:
            viewData = df.to_dict('records')
        
        if len(viewData) == 0:
            return html.Div([
                html.H4('No data to display', style={
                        'textAlign': 'center', 'color': '#95a5a6'})
            ])

        dff = pd.DataFrame.from_dict(viewData)

        # Get outcome type values as a list (not Series) for bucket_categories
        outcome_values = dff['outcome_type'].tolist()

        # Apply category bucketing (top 10 + Other)
        category_mapping = bucket_categories(outcome_values, top_n=10)
        dff['outcome_bucketed'] = dff['outcome_type'].map(category_mapping)

        # Create pie chart
        fig = px.pie(
            dff,
            names='outcome_bucketed',
            title='Outcome Type Distribution',
            color_discrete_sequence=px.colors.qualitative.Set3
        )

        fig.update_traces(textposition='inside', textinfo='percent+label')
        fig.update_layout(showlegend=True, height=500)

        return dcc.Graph(figure=fig)
    except Exception as e:
        print(f"Error in chart callback: {e}")
        return html.Div([
            html.H4('Error loading chart', style={
                    'textAlign': 'center', 'color': '#e74c3c'})
        ])


# Style callback - highlights selected row on current page only
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'derived_virtual_selected_rows')]
)
def highlight_selected_row(derived_virtual_selected_rows):
    """Highlight the selected row with pale green background (current page only)."""
    if derived_virtual_selected_rows is None or len(derived_virtual_selected_rows) == 0:
        return []

    # Use derived_virtual_selected_rows which represents the row index on current page
    return [{
        'if': {'row_index': derived_virtual_selected_rows[0]},
        'backgroundColor': '#D4EDDA',
        'color': '#155724'
    }]


# Map callback - updates geolocation map based on selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    """
    Updates the geolocation map based on selected row in data table.

    Args:
        viewData: Current data visible in the table (filtered/sorted)
        index: List containing index of selected row(s)

    Returns:
        Leaflet map component with marker at selected animal's location
    """
    if viewData is None:
        return
    elif index is None:
        return

    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else:
        row = index[0]

    # Use animal_id to create unique map ID to force remount when row changes
    # This ensures marker position updates even after popup has been opened
    animal_id = str(dff.iloc[row].get('animal_id', row))
    map_key = f"map-{animal_id}"
    
    # Center map on selected animal's location (not fixed Austin coordinates)
    return [
        dl.Map(id=map_key, style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row, 13], dff.iloc[row, 14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]


# Run the app
app.run(jupyter_mode="tab")